<a href="https://colab.research.google.com/github/Xasan6988/chat_bot/blob/main/Chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import random
import nltk

In [37]:
BOT_CONFIG = {
    'intents': {
        'hello': {
            'examples': ['Привет', 'Хай', 'Хэллоу'],
            'responces': ['Здравствуйте', 'Прив', 'Доброго дня'],
        },
        'bye': {
            'examples': ['Пока', 'Увидимся', 'До свидания'],
            'responces': ['Приходите ещё', 'Рад был познакомиться', 'Покеда'],
        },
    }
}

In [38]:
def clean(text):
  output_text = ''
  for char in text.lower():
    if char in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя':
      output_text = output_text + char
  return output_text

def get_intent(text): 
  for intent in BOT_CONFIG['intents']:
    for example in BOT_CONFIG['intents'][intent]['examples']:
      text1 = clean(example)
      text2 = clean(text)
      if len(text1) != 0 and len(text2) != 0:
        if nltk.edit_distance(text1, text2) / max(len(text1), len(text2)) < 0.4:
          return intent
  return 'Не удалось определить интент'

In [39]:
def bot(input_text):
  intent = get_intent(input_text)
  if intent != 'Не удалось определить интент':
    return random.choice(BOT_CONFIG["intents"][intent]['responces'])
  else:
    return 'Извините, я ничего не понял. Я просто глупый бот.'


In [40]:
input_text = ''
while input_text != 'Завершить работу':
  input_text = input()
  print(bot(input_text))


Превед
Прив
Завершить работу
Извините, я ничего не понял. Я просто глупый бот.
